In [26]:
import pandas as pd 
import numpy as np
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from publicsuffix2 import get_public_suffix
from tld import is_tld

In [27]:
import pandas 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [28]:
def check_org (x):
    if x.count('.') == 1:
        return(x.split('.')[0])
    if x.count('.') == 2:
        if is_tld('.'.join(x.split('.')[1:])):return(x.split('.')[0])
        else:return(x.split('.')[1])
    else: return(x)
def check_tld (x):
    if x.count('.') == 1:
        return(x.split('.')[1])
    if x.count('.') == 2:
        if is_tld('.'.join(x.split('.')[1:])):return('.'.join(x.split('.')[1:]))
        else:return(x.split('.')[2])
    else: return(x)
def check_add_info (x):
    if x.count('.') == 2:
        if is_tld('.'.join(x.split('.')[1:])):return('0')
        else:return(x.split('.')[0])
    else: return('0')

In [29]:
# Feature correction 

train_df = pd.read_csv('train.csv', )
test_df = pd.read_csv('test.csv', )
# train_df.drop('Unnamed: 0',axis=1,inplace = True)
# test_df.drop('Unnamed: 0',axis=1,inplace = True)
test_df['label'] = 'test_df'
df = train_df.append(test_df, ignore_index=True)
# df[df ['Unnamed: 0'] == 40]
df['chars_in_subject'] = df['chars_in_subject'].fillna(10)
df.org = df.org.fillna('org_unknown')
df['Ratio-SubtoBody'] = df['chars_in_body'] / (df['chars_in_subject'] + df['chars_in_body'])
df['tld'] = df['tld'].fillna('notfound.com').apply(get_public_suffix)
df['date_tz'] = df['date'].apply(lambda x: x.split('(')[1][:3] if '(' in x else 'not_found')
import datefinder
# df['day'] = df['date'].apply(lambda x:x.split(',')[0])
def find_date(x):
    for x in datefinder.find_dates(x):
        return(x)
        break

df['date'] = df['date'].apply(find_date)
df['date'] = pd.to_datetime(df['date'], utc = True).map(lambda x: x.tz_convert('Asia/Kolkata'))
df['year'] = pd.DatetimeIndex(df['date']).year
df['qtofyr'] = pd.DatetimeIndex(df['date']).quarter
df['weekofyr'] = pd.DatetimeIndex(df['date']).weekofyear
df['month'] = pd.DatetimeIndex(df['date']).month
df['weekday'] = pd.DatetimeIndex(df['date']).dayofweek
df['hour'] = pd.DatetimeIndex(df['date']).hour
df['monthday'] = pd.DatetimeIndex(df['date']).dayofyear
df['second'] = pd.DatetimeIndex(df['date']).hour
df.drop('date',axis=1,inplace=True)
df['working_weekday'] = df['weekday'].apply(lambda x: 1 if x<6 else 0)
df['full_wsite'] = df['org'] +'.'+ df['tld'] 
df['org'] = df['full_wsite'].apply(check_org)
df['tld_2'] = df['full_wsite'].apply(check_tld)
df['add_info'] = df['full_wsite'].apply(check_add_info)

C:\Users\arind\AppData\Local\Temp\ipykernel_33620\3794243463.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train_df.append(test_df, ignore_index=True)
C:\Users\arind\anaconda3\lib\site-packages\publicsuffix2\__init__.py:376: UserWarning: This function returns the private suffix, SLD, or registrable domain. This equivalent to function get_sld(). To get the public suffix itself, use get_tld().
  warnings.warn(
C:\Users\arind\AppData\Local\Temp\ipykernel_33620\3794243463.py:26: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df['weekofyr'] = pd.DatetimeIndex(df['date']).weekofyear


In [30]:
df['work_hour'] = df['hour'].apply(lambda x:1 if (x>9) & (x<17) else 0)
df = pd.get_dummies(df, prefix=['MailType'], columns = ['mail_type'],drop_first=False)
df['tld_front'] = df.tld.apply(lambda x: x.split('.')[0] if '.' in x else '0')
df['i/u_ratio'] = (df['images']+1)/(df['urls'] + 1)
right_df = pd.read_csv('final_label_probab_df.csv')
df = df.merge(right_df, on='org', how='left', indicator=False)
a = df.dtypes.to_frame('type').reset_index()
a[(a['type']=='object') ]['index'].to_list()

['org',
 'tld',
 'label',
 'date_tz',
 'full_wsite',
 'tld_2',
 'add_info',
 'tld_front']

In [31]:
for x in a[(a['type']=='object') ]['index'].to_list(): 
    if x != 'label':
        le_org = preprocessing.LabelEncoder()
        df[x] = le_org.fit_transform(df[x])
df = df.set_index('Unnamed: 0', drop=True)
to_scale = []
for col in df.columns:
    if len(df[col].unique())>2:
        if 'probab' not in col:
            to_scale.append(col)
 

In [32]:
# df.groupby(['add_info', 'label']).count().sort_values('add_info',ascending=False)['org'].to_csv('add_info_check.csv')

In [33]:
to_scale_stsc = [
 'images',
 'urls',
 'chars_in_subject',
 'chars_in_body','Ratio-SubtoBody']

In [34]:
to_scale_nosc = list(set(to_scale) - set(to_scale_stsc))

In [35]:
from sklearn.preprocessing import StandardScaler
for val in to_scale_stsc:
    sc = StandardScaler()
#     df[val] = sc.fit_transform(df[val])
    df[val] = sc.fit_transform(df[[val]])
# sc.transform

In [36]:
for val in to_scale_nosc:
    
#     df[val] = sc.fit_transform(df[val])
    if val!= 'label':
        df[val] = (df[val] - df[val].mean())/df[val].max()

In [37]:
to_scale_nosc

['weekday',
 'hour',
 'tld',
 'i/u_ratio',
 'second',
 'monthday',
 'ccs',
 'label',
 'tld_2',
 'add_info',
 'date_tz',
 'weekofyr',
 'org',
 'year',
 'month',
 'qtofyr',
 'tld_front',
 'full_wsite']

In [38]:
df = df.fillna(1/7)
train_df = df[df['label'] != 'test_df']
train_df['label'] = train_df['label'].astype('int')
test_df = df[df['label'] == 'test_df']

C:\Users\arind\AppData\Local\Temp\ipykernel_33620\2176746312.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['label'] = train_df['label'].astype('int')


In [39]:
train_df.to_csv('To_run_train.csv')
test_df.to_csv('To_run_test.csv')

In [125]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.linear_model import Lasso


In [145]:
df.columns

Index(['org', 'tld', 'ccs', 'bcced', 'images', 'urls', 'salutations',
       'designation', 'chars_in_subject', 'chars_in_body', 'label',
       'Ratio-SubtoBody', 'date_tz', 'year', 'qtofyr', 'weekofyr', 'month',
       'weekday', 'hour', 'monthday', 'second', 'working_weekday',
       'full_wsite', 'tld_2', 'add_info', 'work_hour',
       'MailType_Multipart/Alternative', 'MailType_Multipart/Mixed',
       'MailType_Text/Html', 'MailType_multipart/IDM',
       'MailType_multipart/alternative', 'MailType_multipart/mixed',
       'MailType_multipart/related', 'MailType_multipart/report',
       'MailType_multipart/signed', 'MailType_text/HTML',
       'MailType_text/calendar', 'MailType_text/html', 'MailType_text/html ',
       'MailType_text/plain', 'tld_front', 'i/u_ratio', 'label_0_probab',
       'label_1_probab', 'label_2_probab', 'label_3_probab', 'label_4_probab',
       'label_5_probab', 'label_6_probab', 'label_7_probab'],
      dtype='object')